<center><h1>Exercise on Polygenic Scoring (R Version)</h1>
<b>Systems Genetics Lecture - 25.11.21</b><br><br>
<i><small>Arthur Gilly (arthur.gilly@helmholtz-muenchen.de) - Ana Arruda (ana.arruda@helmholtz-muenchen.de)</small></i>
</center>

# Summary
In this exercise, we will apply two polygenic scores to a cohort of 1000 indivuduals. The samples come from the 1000 Genomes project, and the two scores are **a polygenic risk score for Coronary Artery Disease (CAD)** and a **polygenic score for levels of the MEP1B protein**. We will study the influence of ethnicity, see how well these two scores predict the traits in question, and finally, we will examine the polygenicity of these two traits through a genome-wide association.

## Downloading the data and installing libraries

<div class="alert alert-warning">This R notebook uses the <code>data.table</code> library for data manipulation and base R for graphics. In particular, <code>magrittr</code>, <code>ggplot2</code> and <code>tidyr</code> are not supported.</div>

All the libraries you need are already installed and most data files are in the `data` directory.



## Lifting over the polygenic score

We will be working with a CAD score that was downloaded from the publicly available PGS catalog. Variants in that score are identified by chromosome:position, but are on build 37 (also called hg19) of the human reference genome. Our genetic data is on build 38, we must therefore first map these coordinates onto that build. That process is called a liftover. There are R libraries to do this, but they come from the Bioconductor project, and are clunky and hard to use. We will use an external program called CrossMap.

First, we download the dictionary of positions from build 37 to 38 from the USCS Liftover FTP Website.

**Question 1**: Download the chain file from https://hgdownload.cse.ucsc.edu/goldenpath/hg19/liftOver/hg19ToHg38.over.chain.gz

In [ ]:
cat(system('
## Paste your command here followed by : 2>&1
', intern=T), sep="\n")


Here is CrossMap's help:

In [ ]:
cat(system('
CrossMap.py --help 2>&1
', intern=T), sep="\n")


As you can see, it supports multiple input formats. We are going to use `bed` as it is the easiest to use. This format is composed of 3 mandatory, tab-separated columns, and an arbitrary number of columns afterwards. The 3 columns are `chr    pos-1    pos`. 


**Question 2:** Read in the CAD score (`data/PGS000337.txt.gz`) and make it compatible with the BED format.

In [ ]:
library(data.table)
# use function fread

In [ ]:
# create a new column subtracting 1 from the position
# reorder columns if necessary
# use function fwrite to write file to disk, use tab, do not print header, do not use quotes

**Question 3:** Use `CrossMap bed` to convert the positions in the CAD score from build 37 to build 38.

In [ ]:
cat(system('
CrossMap.py bed --help 2>&1
', intern=T), sep="\n")


**Question 4:** Read in the lifted over file, remove the second column (`pos - 1`), remove any position that maps outside of the autosomes (chr1-chr22), and check that no variant maps to several positions on build 38.

In [ ]:
# DT[,col:=NULL] will remove column "col"
# use setnames to rename columns
# use %in% to check whether members of a vector contains values from another
# use function table to count occurrences of an element in a vector, or DT[,.N, by=col]

**Question 5:** Create a tab-separated, headerless score file for your lifted over CAD score, with the following columns :
* `id` which has the form chr1:1234
* `effect_allele`
* `effect_weight`

In [ ]:
# use paste and paste0 to concatenate columns

**Question 6:** Do the same for the MEP1B protein score located at `data/MEP1B.gilly.prs.txt`. The coordinates in that file are already in build 38, you only need to reformat the file, not lift it over.

## Applying scores

We will now apply the scores to a cohort of 1,000 individuals. From the lecture, you will remember that we need genotype data for these individuals as well as the weights contained in the score files above. We will first use PLINK, a popular genetic toolbox installed in this environment. Then we will do it manually, and compare the results.

### Method 1 : using PLINK

**Question 7:** Check the PLINK 1.9 user manual for applying risk scores and create two score profiles. Use the `data/autosomal.forPRS.{bed|bim|fam}` dataset for genotypes and the score files you have created in questions 6 and 7.

In [ ]:
cat(system('
plink --help
', intern=T), sep="\n")

#### Visualising scores

**Question 8:** Visualise the distributions of scores for the two scores you have just calculated. Comment on the distribution shape for both scores.

In [ ]:
# use the 'hist' function, increase bins by using the 'breaks' argument

### Method 2: Manually applying scores (bonus section, only if you are proficient with data.table and/or data wrangling in R)

We do not exactly know what PLINK did for us. We can repeat the process manually using R data.table's data manipulation tools.

**Question 9:** Load the `autosomal.forPRS.mx.traw` file. It is a matrix of genotypes with positions as rows and samples as columns. There are 7 "header" columns describing each position. For each score, restrict the dataframe to the positions in the score file, then apply an element-wise multiplication column by column, and sum the weighted genotypes: $score_j=\sum_{i \in SNPs}w_i*g_i$ where $i$ denotes SNPs and $j$ denotes individuals. For the CAD score, beware that the score file and genotype file do not contain the same variants, and that alleles may be different even if they are present.

In [ ]:
# read the traw
# read the famfile
# subset the traw
# create a new DF with the subset of SNPs in the MEP1B score
# apply the score using lapply and colSums

<div class="alert alert-warning"> You might be thinking of <code>cadg=genos[SNP %in% cadscore$id]</code> which usually works. But this binder has very low memory, so a merge is more efficient. Don't forget to <code>gc()</code> from time to time to free up memory.

In [ ]:
# merge the cad score DF with the traw DF and delete the traw DF from memory

In [ ]:
# compare alleles from the score to the COUNTED allele in the traw. Identical ones are fine.
# for discordant ones, you can either discard them or try to match the ALT allele and flip the effects

**Question 10:** Merge all scores into a single dataframe, and compute the correlations between your scores and the ones calculated by PLINK. Where do you think the difference comes from?

In [ ]:
# use merge and cor

## Predictive accuracy of MEP1B levels and CAD events

**Question 11:** Load the CAD and MEP1B phenotypes for these individuals in `data/CAD.phenotype` and `data/MEP1B.phenotype`. Compute the Pearson's correlation for MEP1B, and the odds ratio for people in the top vs bottom deciles of the distribution for CAD. What can you say about the predictive accuracy of both scores?


In [ ]:
# For MEP1B, merge and cor again, you can plot(x,y) to produce a scatterplot

In [ ]:
# For CAD, compute score quantiles using `quantile`, and check number of C/c using table.
# then calculate the odds ratio using simple arithmetic

## PRS and Polygenicity

Until now, we have applied two genetic risk scores and examined how well they predict actual phenotypes. We will now examine what these scores can tell us about the genetic architecture of these traits.

**Question 12:** How many variants are present in each score? Does that correspond to what you know of the genetic architecture of both traits?

In [ ]:
#nrow or dim

MEP1B is a protein trait, expected to be much less polygenic than CAD, a complex trait.

**Question 13:** Perform a genome-wide association, using each score profile as a phenotype (you will need to create a properly formatted `.pheno` file) and `PRS.course.testset` as a binary genotype set, using PLINK's `--assoc` flag. You can use `manqq::fastqq(P)` to visualise the results. What can you say about these association results?

In [ ]:
# plink wants FID IID and pheno as columns, no header.
# tstrsplit may be useful to create 2 columns from a split


**Question 16:** This cohort is multiethnic, and there is therefore a strong chance that some score variants will be correlated with ethnicity. Use the genetic principal components from `data/PCs.eigenvec` and use them as covariates using the `--linear` and `--covar` flags of PLINK. Check that your association results are now well-behaved. You can use `manqq::fastmanh(DT, no_annot=T)` to visualise results, where `DT` has three columns: `chr`, `pos` and `p`.

<div class="alert alert-warning">Running associations in linear mode can be quite slow (~10 minutes).</div>


In [ ]:
# be careful to remove NAs and 0s from the P-values in your DF

**Question 17:** What can you deduce from the two manhattan plots about the architecture of these two traits?